In [ ]:
import numpy as np
import copy

import random

from py4j.java_gateway import JavaGateway
from collections import deque

import pad_discriminator_conformance

import statistics

In [ ]:
trace1 = ['A', 'B', 'D', 'E', 'I']
trace2 = ['A', 'C', 'D', 'G', 'H', 'F', 'I']
trace3 = ['A', 'C', 'G', 'D', 'H', 'F', 'I']
trace4 = ['A', 'C', 'H', 'D', 'F', 'I']
trace5 = ['A', 'C', 'D', 'H', 'F', 'I']


traces = [trace1, trace2, trace3, trace4, trace5]
frequencies = [1207, 145, 56, 23, 28]

GT_log = []
for f in range(0, len(frequencies)):
    print(traces[f], frequencies[f])
    for i in range(0, frequencies[f]):
        GT_log.append(traces[f])
        
random.shuffle(GT_log)

In [ ]:
class OutputGetter(deque):
    def appendleft(self, line):
        print(line)
        super().appendleft(line)
        
def remove_stuff(log): #removes short traces
    #new_log = [i for i in log if len(i) > 5]
    new_log = [i for i in log if len(i) > 1]
    return new_log
    
        
def remove_plus(dummylog):
    newlog = copy.deepcopy(dummylog)
    newlog = [[item.replace('+', '') for item in lst] for lst in newlog]
    return newlog
       
def fix_duplicates(dummylog):
    newlog = []
    for trace in dummylog:
        newtrace = trace[::2]
        newlog.append(newtrace)
    return newlog

def check_duplicates(dummylog): #not a good test but just to be sure
    if len(dummylog) > 1 and dummylog[0][0] == dummylog[0][1] and dummylog[1][0] == dummylog[1][1]:
        return True
    elif len(dummylog) == 1 and dummylog[0][0] == dummylog[0][1]:
        return True
    else:
        return False

In [ ]:
def get_model_log(modelfilename, size):
    gateway = JavaGateway.launch_gateway(classpath="./jars/*", 
                                         redirect_stdout=OutputGetter())
    petri_and_marking = gateway.jvm.org.processmining.plugins.kutoolbox.utils.ImportUtils.openPetrinet(
        gateway.jvm.java.io.File(modelfilename)
    )
    if not petri_and_marking[1].size():
        #print("Creating initial marking")
        petri_and_marking[1] = gateway.jvm.org.processmining.plugins.kutoolbox.utils.PetrinetUtils. \
            getInitialMarking(petri_and_marking[0])

    petri_and_marking[1].size()
    settings = gateway.jvm.org.processmining.plugins.loggenerator.utils.GeneratorSettings()
    simulator = gateway.jvm.org.processmining.plugins.loggenerator.utils.GeneratorSettings.SimulationAlgorithm

    # See https://github.com/Macuyiko/processmining-prom/blob/master/loggenerator/
    #             org/processmining/plugins/loggenerator/utils/GeneratorSettings.java

    for t in petri_and_marking[0].getTransitions().iterator():
        isInvisible = t.getLabel() == "" or t.isInvisible()
        label = t.getLabel()
        isTextInvisible = label.startswith("inv_") or "$" in label
        mapped = label.replace("+complete", "").replace("\\ncomplete", "").replace("\\n", "") \
            if not isInvisible and not isTextInvisible else ""
        arr = gateway.new_array(gateway.jvm.int, 4)
        arr[0], arr[1], arr[2], arr[3] = 60, 0, 60, 0
        settings.getTransitionNames().put(t, mapped)
        settings.getTransitionWeights().put(t, 10)
        settings.getTransitionTimings().put(t, arr)
        #print(t.getId(), ':', label, '--->', mapped)

        settings.setNrTraces(size)
        settings.setRandomMinInGroup(1)
        settings.setRandomMaxInGroup(1)
        settings.setMustReachEnd(True)
        settings.setMustConsumeAll(False)
        settings.setMaxTimesMarkingSeen(7)

        settings.setSimulationMethod(simulator.Random) # Random, Complete, or Distinct

        # settings.setRestartAfter(1000)
        # settings.setSkipChance(0.85)
        # settings.setAlsoConsiderPartial(False)
    xlog = gateway.jvm.org.processmining.plugins.loggenerator.PetriNetLogGenerator.generate(
        petri_and_marking[0], 
        petri_and_marking[1], 
        settings, 
        #None # or for output: gateway.jvm.org.processmining.plugins.kutoolbox.eventlisteners.PluginEventListenerCLI()
        gateway.jvm.org.processmining.plugins.kutoolbox.eventlisteners.PluginEventListenerCLI()
    )

    temp_log = []
    for trace in xlog:
        #print([gateway.jvm.org.deckfour.xes.extension.std.XConceptExtension.instance().extractName(e) for e in trace])
        temp_log.append([gateway.jvm.org.deckfour.xes.extension.std.XConceptExtension.instance().extractName(e) for e in trace])

    model_log = remove_stuff(temp_log)


    if check_duplicates(model_log) == True:
        #print('Fixing duplicates')
        model_log = fix_duplicates(model_log)
    return(model_log)

In [ ]:
def over_time_test(GT_log, filename, times, modellogsizes): #random model log generation
    fitness_count = []
    fitness_prob = []
    precision_count = []
    precision_prob = []
    
    fitness_count_errors = []
    fitness_prob_errors = []
    precision_count_errors = []
    precision_prob_errors = []
    
    for t in range(0,len(modellogsizes)):
        fitness_results1 = []
        precision_results1 = []
        fitness_results2 = []
        precision_results2 = []

        for i in range(0, times):
            print("step", modellogsizes[t], " and ", i, " OUT OF ", times)
            model_log = get_model_log(filename, modellogsizes[t])
            fitness1, fitness2 = pad_discriminator_conformance.get_distance(GT_log, model_log, "random")
            precision1, precision2 = pad_discriminator_conformance.get_distance(model_log, GT_log, "random")
            fitness_results1.append(fitness1)
            fitness_results2.append(fitness2)
            precision_results1.append(precision1)
            precision_results2.append(precision2)
        fitness_prob.append(statistics.median(fitness_results1))
        fitness_count.append(statistics.median(fitness_results2))
        precision_prob.append(statistics.median(precision_results1))
        precision_count.append(statistics.median(precision_results2))
        fitness_prob_errors.append(np.std(fitness_results1)/np.sqrt(10))
        fitness_count_errors.append(np.std(fitness_results2)/np.sqrt(10))
        precision_prob_errors.append(np.std(precision_results1)/np.sqrt(10))
        precision_count_errors.append(np.std(precision_results2)/np.sqrt(10))
        print(fitness_count)
        print(fitness_prob)
        print(fitness_count_errors)
        print(fitness_prob_errors)
        print(precision_count)
        print(precision_prob)
        print(precision_count_errors)
        print(precision_prob_errors)
    return(fitness_prob, fitness_count, precision_prob, precision_count, fitness_prob_errors, fitness_count_errors, precision_prob_errors, precision_count_errors)
        
   

In [ ]:
#sizes = [5, 10, 20, 50, 100, 200, 300, 400, 500]

#sizes = [600, 700, 800, 900]

#sizes = [5, 20, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900]

sizes = [400, 500, 600, 700, 800, 900]

#sizes = [100, 500, 1000, 1500, 2000, 2500, 3000]


In [ ]:
fit1, fit2, prec1, prec2, errorfit1, errorfit2, errorprec1, errorprec2 = over_time_test(GT_log, "Model11.pnml", 10, sizes)

In [ ]:
sizes = [5, 20, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900]

In [ ]:

fit1 = [0.5811467170715332, 0.5769599676132202, 0.5001965165138245, 0.49010172486305237, 0.4322972297668457, 0.18222317099571228] + fit1
fit2 = [0.9712131596984236, 1.0, 0.17272104180945852, 0.17272104180945852, 0.17272104180945852, 0.17272104180945852]  + fit2
errorfit1 = [0.02402501604127421, 0.013840682734899435, 0.03589896414823458, 0.055182698119664454, 0.06740481837234095, 0.06813614103424814] + errorfit1
errorfit2 = [0.12419160697822577, 0.0879108854670524, 0.12816150510959154, 0.11988411057334236, 0.11988411057334236, 0.08008022544110256]  + errorfit2

prec1 = [0.9973577260971069, 0.9962339401245117, 0.9887729287147522, 0.9982422590255737, 0.998394250869751, 0.9975321292877197] + prec1
prec2 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0] + prec2
errorprec1 =[0.0032716706996467376, 0.0038220030894246418, 0.013988596287993212, 0.009176445085041797, 0.003450074289943527, 0.0067207795856227244]  + errorprec1
errorprec2 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0] + errorprec2


In [ ]:
print(fit1,fit2,errorfit1,errorfit2,prec1,prec2,errorprec1,errorprec2)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#plt.errorbar(sizes, fit1, errorfit1, label="Recall Prob.", ecolor = 'r', barsabove=True)
#plt.errorbar(sizes, fit2, errorfit2, label="Recall Count", color= 'g', ecolor = 'r', barsabove=True)
plt.errorbar(sizes, prec1, errorprec1, label="Precision Probability",  color= 'y', ecolor = 'r', barsabove=True)
plt.errorbar(sizes, prec2, errorprec2, label="Precision Count",  color= 'm', ecolor = 'r', barsabove=True)
plt.xlabel('Size Model Log')
plt.ylabel('Precision')
plt.legend()
plt.title("Prec. Model 11 with increasing model log size")
plt.savefig("convergence_model11_rec.pdf")
plt.show()